 **Damage Mechanics**
==================================================================================================
Work 1 - Isotropic plasticity with linear hardening
by Guilherme Jenovencio, M. Sc. 
--------------------------------------------------------
 
Model Description:
-  Additive decomposition of the strain tensor:

$$\varepsilon = \varepsilon^e + \varepsilon^p \tag{1} $$                  

- Definition of the Free-energy function 

$$\Psi(\varepsilon^e,r) =   \Psi^e(\varepsilon^e) + \Psi^p(r) \tag{2} $$  

Where r is the accumulated platic strain: $r = \int_{0}^{t}|{\dot{\varepsilon^p}}|dt  $.

- Constitutive equation for $\sigma$ and thermodynamic force R 

$$\sigma =  \frac{\partial \Psi^e}{\partial \varepsilon^e} \tag{3} $$  
 
$$R =  \frac{\partial \Psi^p}{\partial r} \tag{4} $$   

- Yield function 

$$f =  f(\sigma,R) \tag{5} $$   

-  Dissipation potential 

$$g =  f(\sigma,R) \tag{6} $$   

- Plastic flow rule and hardening law 
$$ \dot{\varepsilon^p} = \dot{\lambda}\frac{\partial f}{\partial \sigma} = \dot{\lambda}\mathbf{N} \tag{7} $$   

$$ \dot{r} = -\dot{\lambda}\frac{\partial f}{\partial R} = \dot{\lambda}\mathbf{H}  \tag{8} $$   


- Loading/unloading criterion 

$$f(\sigma,R) \leq 0 \tag{9} $$   

$$\dot{\lambda} \geq 0 \tag{10} $$

$$f(\sigma,R)\dot{\lambda} = 0 \tag{11}$$  

-  Consistency condition under plastic yielding $\dot{\lambda}\ne0$

$$\dot{f(\sigma,R)}\dot{\lambda} = 0 \tag{12}$$  


Selection of a free energy potencial and the Yield function:

- Free energy potencial
$$\Psi^e(\varepsilon^e) = + \frac{1}{2}\mathbf{D}\varepsilon^e.\varepsilon^e \tag{13} $$  

$$\Psi^p(r) = + \frac{1}{2}Hr^2 \tag{14} $$  

Where **D** is a forth order tensor and *H* is a scalar parameter are both material properties. 

- Yield function
$$f =  f(\sigma,R) = \sigma_{eq}^D - \sigma_y(r) \tag{15} $$ 

Where 

$$ \sigma = \sigma^D + \frac{tr(\sigma)}{3}I = \sigma^D + pI \tag{16} $$

$$ \sigma_{eq}^D = (\frac{3}{2} \sigma^D.\sigma^D)^\frac{1}{2} \tag{17} $$

$$\sigma_y(r) = \sigma_{yo}(r) +R(r) = \sigma_{yo}(r) + Hr \tag{18} $$ 



**Elastic predictor / Return-Mapping Algorithm**
===============================================

Initial Constitutive Value problem

Let's say the total deformation tensor at the current time $\varepsilon_{n+1}$ and the state variable at the previous time $\alpha_n$ are known, then we will determine the stress and the state variable vector ($\sigma_{n+1}$, $\alpha_{n+1}$) at the current time, where :



$$\alpha = \{r ,\varepsilon^p \} $$


Integration (7) implicity we get the folowwing expression for the plastic strain increment:

$$ \Delta{\varepsilon^p} = \Delta{\lambda}\mathbf{N} \tag{19} $$  

where $\Delta{\lambda}$ is the plastic multiplier increment.

- Elastic trial.


Firstly, we assume that the strain increment $\Delta{\varepsilon}$ is elastic  , which also means $\Delta{\lambda}=0$.
Then we set the inicial conditions as:

$$\alpha_{n+1}^{trial} = \alpha_{n} \tag{20} $$


$$ \varepsilon_{n+1}^{etrial} = \varepsilon_{n+1} - \varepsilon_n^p \tag{21}  $$

The deviatoric and volumetric split can be aplied on (21):

$$ \varepsilon_{n+1}^{etrial} = \hat{\varepsilon}_{n+1}^{etrial} + {\varepsilon}_{vol [n+1]}^{etrial} \tag{22} $$

Where ${\varepsilon}_{vol [n+1]}^{etrial} = \frac{1}{3}tr(\varepsilon_{n+1}^{etrial})I $

Then, (16) can be written in terms of the trial strain as:

$$ p^{trial} = K{\varepsilon}_{vol [n+1]}^{etrial} \tag{23}$$

$$ \sigma_{eq}^{Dtrial} = 2G\hat{\varepsilon}_{n+1}^{etrial} \tag{24} $$


Then, the von-Mises stress (17) can be writen as:

$$ \sigma_{eq}^{Dtrial} = \sqrt{\frac{3}{2} \sigma^D.\sigma^D} \tag{25} $$


The yield function can be checked with (25) and (15):

$$f^{trial}  = \sigma_{eq}^{Dtrial} - (\sigma_{yo}+Hr_n) <0 \tag{26} $$ 

if (26) is true then the stress lies in the yield function, in other case the plastic multiplier increment should be greater then zero. 

Implementation of Elastic trial
=========================


In [46]:
import numpy as np

def matprop():
    ''' This fucntion will return the  material properties
    '''
    E = 200.0e3 # Elastic modulus in MPa
    v = 0.3 # poisson ration
    G = E/(2.*(1. + v))  # Shear modulus
    K = E/(3.*(1. - 2.*v)) # Bulk modulus
    
    sigmayo = 200. # initial yield stress in MPa
    H = 5.0e3 # Hardening modulus in MPa
    return E,v,G,K,sigmayo,H
    
def initState(rn=0,epn=np.zeros((3,3))):
    ''' This function will return the initial internal variables
    '''
    rn = rn # acculumated plastic strain
    epn = epn # plastic strain at n
    
    return rn, epn

def dhdec(S):
    ''' deviatoric and hidrostatic decomposition
    given a tensor  S split in Sdev and svol*I
    where Sdev is de deviatoric part
    and Svol is the 
    '''
    svol = np.trace(S)
    Sdev = S - svol/3.0
    
    return Sdev, svol

def yieldfunc(sdev,rn):
    ''' this is the yield function
        sdev  is de deviatoric stress
        rn is the accumulated plastic strain
    '''
    
    E,v,G,K,sigmayo,H = matprop()
    
    vm = np.sqrt(3.0*np.tensordot(sdev,sdev)/2.0)
    
    print('von-Mises stress = %f' %vm)
    
    Rtriel = sigmayo + H*rn
    f =  vm - Rtriel
    return f
    
def elasticTrial(En1,an):
    '''This function the get the elastic strain at n + 1 and 
    the state variable in nand check is the stress is inside the yield surface.
    ''' 
    # getting material properties
    E,v,G,K,sigmayo,H = matprop()
    
    rn, epn = an
    # elastic triel
    eetrial = En1 - epn #elastic triel
    eetrieldev, eetrialvol = dhdec(eetrial)
    
    sdevtrial = 2.0*G*eetrieldev # deviatoric stress
    
    f = yieldfunc(sdevtrial,rn)

    if f<0:
        return True
    else: 
        return False
    
#----------------------------------------------------------------------------
# Test elasticTrial function

inc =0.0001 # displacement increment   
n = 14      # number of displaciment increment

exx = n*inc
eyy = -n*0.5*inc
ezz = eyy

En1 = np.matrix([ [exx ,  0.    ,   0.   ], 
                  [0.  , eyy    ,   0.   ],
                  [0.  ,  0.    ,    ezz ] ])


p = 0. # proportion of plastic strain
epn = p*En1


# getting the internal variable
rn = np.sqrt(3.0*np.tensordot(epn,epn)/2.0)
an = initState(rn,epn)

iselastic = elasticTrial(En1,an)

if iselastic:
    print("The strain increment is pure elastic")
else:    
    print("The strain increment is NOT pure elastic. The plastic multiplier should be greater than 0." +
          "\nGo to Return Mapping algorithm.")


von-Mises stress = 323.076923
The strain increment is NOT pure elastic. The plastic multiplier should be greater than 0.
Go to Return Mapping algorithm


(0, array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]]))


0.999985